# Geometrical Methods in Machine Learning

## Seminar 4: Nonlinear Dimensionality Reduction

In [ ]:
!pip install --upgrade sklearn
!pip install --upgrade grakel
!pip install --upgrade POT

In [ ]:
from __future__ import print_function

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import offsetbox

import numpy as np

from sklearn.datasets import load_digits, fetch_olivetti_faces, make_moons, make_circles, make_swiss_roll, fetch_openml
from sklearn.metrics.pairwise import linear_kernel, rbf_kernel, sigmoid_kernel, cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.manifold import MDS, Isomap, LocallyLinearEmbedding, SpectralEmbedding

from grakel import GraphKernel
from grakel.kernels import VertexHistogram, ShortestPath, WeisfeilerLehman
from grakel.datasets import fetch_dataset

import ot

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## 1. Kernel PCA

In [ ]:
# load data
X, y = make_moons(n_samples=500, noise=0.01, random_state=42)
X_std = StandardScaler().fit_transform(X)

In [ ]:
plt.figure(figsize=(8,5))
plt.scatter(X_std[y==0, 0], X_std[y==0, 1], color='red', alpha=0.25)
plt.scatter(X_std[y==1, 0], X_std[y==1, 1], color='blue', alpha=0.25)
plt.title('A nonlinear 2D dataset')
plt.ylabel('Y')
plt.xlabel('X')
plt.grid(linestyle="dotted")
plt.show()

In [ ]:
# apply PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

pca_1d = PCA(n_components=1)
X_pca_1d = pca_1d.fit_transform(X)

In [ ]:
plt.figure(figsize=(16,4.5))

plt.subplot(131)
plt.title('Original data')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(linestyle="dotted")
plt.scatter(X[y==0, 0], X[y==0, 1], color='red', alpha=0.25)
plt.scatter(X[y==1, 0], X[y==1, 1], color='blue', alpha=0.25)

plt.subplot(132)
plt.title('PCA')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid(linestyle="dotted")
plt.scatter(X_pca[y==0, 0], X_pca[y==0, 1], color='red', alpha=0.25)
plt.scatter(X_pca[y==1, 0], X_pca[y==1, 1], color='blue', alpha=0.25)

plt.subplot(133)
plt.title('Projection to PC1')
plt.xlabel('PC1')
plt.grid(linestyle="dotted")
plt.scatter(X_pca_1d[y==0, 0], np.zeros((int(X_pca.shape[0]/2),1)), color='red', alpha=0.25)
plt.scatter(X_pca_1d[y==1, 0], np.zeros((int(X_pca.shape[0]/2),1)), color='blue', alpha=0.25)

plt.show()

## 1.1 Kernels

Kernel is any symmetric positive-definite function of the form:

$$K(\mathbf{x}, \mathbf{x}') = \langle \phi(\mathbf{x}), \phi(\mathbf{x}') \rangle$$

where $\phi$ is a function that projects vectors $\mathbf{x}$ and $\mathbf{x}'$ into a new vector space. The kernel function can be seen as a function that computes the inner-product between two vectors in some, potentially infinite-dimensinal space.

### 1.1.1 Linear kernel

A _linear kernel_ is given by $\mathbf{x}^T\mathbf{x}$, thus the inner product in the original vector space. Consider a $n \times m$ dataset $\mathbf{X} = \{ \mathbf{x}_1, \dots, \mathbf{x}_n\}$ with $n$ observations and $m$ features, then _Gram matrix_ $\mathbf{G} = \mathbf{X} \mathbf{X}^T$ is a matrix of inner-products, in this notation covariance matrix $\mathbf{S}$ is given by $\mathbf{X}^T\mathbf{X}$.

#### Exercise

Compute the matrix of pairwise linear kernels for a moons dataset $\mathbf{X}$, check that it is Gram matrix. You can use `linear_kernel` from `sklearn.metrics.pairwise`.

In [ ]:
K = # your code here
G = # your code here

K, G

Kernel PCA with linear kernel obviously is equivalent to PCA.

In [ ]:
# apply Kernel PCA with linear kernel
kpca = KernelPCA(n_components=2)
X_kpca = kpca.fit_transform(X)

kpca_1d = KernelPCA(n_components=1)
X_kpca_1d = kpca_1d.fit_transform(X)

In [ ]:
plt.figure(figsize=(16,4.5))

plt.subplot(131)
plt.title('Original data')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(linestyle="dotted")
plt.scatter(X[y==0, 0], X[y==0, 1], color='red', alpha=0.25)
plt.scatter(X[y==1, 0], X[y==1, 1], color='blue', alpha=0.25)

plt.subplot(132)
plt.title('Kernel PCA /w linear kernel')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid(linestyle="dotted")
plt.scatter(X_kpca[y==0, 0], X_kpca[y==0, 1], color='red', alpha=0.25)
plt.scatter(X_kpca[y==1, 0], X_kpca[y==1, 1], color='blue', alpha=0.25)

plt.subplot(133)
plt.title('Projection to PC1')
plt.xlabel('PC1')
plt.grid(linestyle="dotted")
plt.scatter(X_pca_1d[y==0, 0], np.zeros((int(X_pca.shape[0]/2),1)), color='red', alpha=0.25)
plt.scatter(X_pca_1d[y==1, 0], np.zeros((int(X_pca.shape[0]/2),1)), color='blue', alpha=0.25)

plt.show()

### 1.1.2 Gaussian kernel

A radial basis function (RBF) is a real-valued function whose value depends only on the distance $d_X$ from some other point $x'$, so that $\phi (\mathbf {x} ,\mathbf {x'} )= \phi (\|\mathbf {x} -\mathbf {x'} \|)$.

Many distances $d_X$ induce radial basis kernels which given by:

$$K(\mathbf{x}, \mathbf{x}') = \exp \left(- \gamma \| \mathbf{x} -\mathbf{x}' \|^2\right)$$

 If $\gamma = \frac{1}{2 \sigma^2}$ it is known as the Gaussian kernel:

$$K(\mathbf{x}, \mathbf{x}') = \exp \left(- \frac{\| \mathbf{x} -\mathbf{x}' \|^2 }{2 \sigma^2}\right)$$

#### Exercise

Compute the matrix of pairwise RBF kernels for a dataset $\mathbf{X}$. Implement Gaussian kernel, check whether it values are equals scikit-learn solution. Check kernel values for different $\gamma$ and $\sigma$. You can use `rbf_kernel` from `sklearn.metrics.pairwise`.

In [ ]:
# compute K for X, matrix of paiwise RBF kernels for dataset X
K = # your code here
K

In [ ]:
# implement Gaussian kernel
def gaussian_kernel(x, x_prime, sigma=1):
    return # your code here

#### Exercise

Apply Kernel PCA with Gaussian kernel with different values of $\gamma$ to Moons dataset.

In [ ]:
# apply KernelPCA with Gaussian kernel
gamma = 20

kpca = KernelPCA(n_components=2, kernel="rbf", gamma=gamma)
X_kpca = kpca.fit_transform(X)

kpca_1d = KernelPCA(n_components=1, kernel="rbf", gamma=gamma)
X_kpca_1d = kpca_1d.fit_transform(X)

In [ ]:
plt.figure(figsize=(16,4.5))

plt.subplot(131)
plt.title('Original data')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(linestyle="dotted")
plt.scatter(X[y==0, 0], X[y==0, 1], color='red', alpha=0.25)
plt.scatter(X[y==1, 0], X[y==1, 1], color='blue', alpha=0.25)

plt.subplot(132)
plt.title('Kernel PCA /w Gaussian kernel')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid(linestyle="dotted")
plt.scatter(X_kpca[y==0, 0], X_kpca[y==0, 1], color='red', alpha=0.25)
plt.scatter(X_kpca[y==1, 0], X_kpca[y==1, 1], color='blue', alpha=0.25)

plt.subplot(133)
plt.title('Projection to PC1')
plt.xlabel('PC1')
plt.grid(linestyle="dotted")
plt.scatter(X_kpca_1d[y==0, 0], np.zeros((int(X_kpca.shape[0]/2),1)), color='red', alpha=0.25)
plt.scatter(X_kpca_1d[y==1, 0], np.zeros((int(X_kpca.shape[0]/2),1)), color='blue', alpha=0.25)

plt.show()

### 1.1.3 Polynomial kernel

The polynomial kernel is defined as:

$$K(\mathbf{x}, \mathbf{x'}) = (\gamma \mathbf{x}^\top \mathbf{x}' + c)^d$$

where $\mathbf{x}, \mathbf{x'}$ are the input vectors, $\gamma$, $c$ are scalars and $d$ is the kernel degree.

#### Exercise

Apply KernelPCA with polynomial kernel to Moons dataset.

Check how standartization affects embedding with polynomial kernel.

In [ ]:
# standartize the data
X_std = StandardScaler().fit_transform(X)

In [ ]:
# apply KernelPCA with Gaussian kernel
degree = 3

kpca = KernelPCA(n_components=2, kernel="poly", degree=degree)
X_kpca = kpca.fit_transform(X_std)

kpca_1d = KernelPCA(n_components=1, kernel="poly", degree=degree)
X_kpca_1d = kpca_1d.fit_transform(X_std)

In [ ]:
plt.figure(figsize=(16,4.5))

plt.subplot(131)
plt.title('Original data')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(linestyle="dotted")
plt.scatter(X[y==0, 0], X[y==0, 1], color='red', alpha=0.25)
plt.scatter(X[y==1, 0], X[y==1, 1], color='blue', alpha=0.25)

plt.subplot(132)
plt.title('Kernel PCA /w polynomial kernel')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid(linestyle="dotted")
plt.scatter(X_kpca[y==0, 0], X_kpca[y==0, 1], color='red', alpha=0.25)
plt.scatter(X_kpca[y==1, 0], X_kpca[y==1, 1], color='blue', alpha=0.25)

plt.subplot(133)
plt.title('Projection to PC1')
plt.xlabel('PC1')
plt.grid(linestyle="dotted")
plt.scatter(X_kpca_1d[y==0, 0], np.zeros((int(X_kpca.shape[0]/2),1)), color='red', alpha=0.25)
plt.scatter(X_kpca_1d[y==1, 0], np.zeros((int(X_kpca.shape[0]/2),1)), color='blue', alpha=0.25)

plt.show()

### 1.1.4. Cosine similarity

Cosine similarity is the normalized dot product between vectors $\mathbf{x}, \mathbf{x}'$ which is the cosine of the angle between the points denoted by the vectors:

$$K(\mathbf{x}, \mathbf{x}') = \frac{\mathbf{x} \mathbf{x}'^\top}{\|\mathbf{x}\| \|\mathbf{x}'\|}$$

#### Exercise

Apply Kernel PCA with cosine similarity `cosine_similarity` as precomputed kernel to Moons dataset.

Check how standartization affects embedding with cosine similarity kernel.

In [ ]:
# standartize the data
X_std = StandardScaler().fit_transform(X)

In [ ]:
# compute K, matrix of paiwise cosine similarities of dataset X
K = cosine_similarity(X)

In [ ]:
# apply KernelPCA with cosine similarity as precomputed kernel
kpca = KernelPCA(n_components=2, kernel="precomputed")
X_kpca = kpca.fit_transform(K)

kpca_1d = KernelPCA(n_components=1, kernel="precomputed")
X_kpca_1d = kpca_1d.fit_transform(K)

In [ ]:
plt.figure(figsize=(16,4.5))

plt.subplot(131)
plt.title('Original data')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(linestyle="dotted")
plt.scatter(X_std[y==0, 0], X_std[y==0, 1], color='red', alpha=0.25)
plt.scatter(X_std[y==1, 0], X_std[y==1, 1], color='blue', alpha=0.25)
#plt.scatter(X[y==0, 0], X[y==0, 1], color='green', alpha=0.25)
#plt.scatter(X[y==1, 0], X[y==1, 1], color='orange', alpha=0.25)

plt.subplot(132)
plt.title('Kernel PCA /w cosine similarity kernel')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid(linestyle="dotted")
plt.scatter(X_kpca[y==0, 0], X_kpca[y==0, 1], color='red', alpha=0.25)
plt.scatter(X_kpca[y==1, 0], X_kpca[y==1, 1], color='blue', alpha=0.25)

plt.subplot(133)
plt.title('Projection to PC1')
plt.xlabel('PC1')
plt.grid(linestyle="dotted")
plt.scatter(X_kpca_1d[y==0, 0], np.zeros((int(X_kpca.shape[0]/2),1)), color='red', alpha=0.25)
plt.scatter(X_kpca_1d[y==1, 0], np.zeros((int(X_kpca.shape[0]/2),1)), color='blue', alpha=0.25)

plt.show()

### 1.2 Graph kernels

In [ ]:
# load graph dataset
mutag = fetch_dataset("PROTEINS", verbose=False)
X_mutag, y_mutag = mutag.data, mutag.target
y_mutag.shape

In [ ]:
y_mutag

In [ ]:
# intitalize Weisfeiler-Lehman kernel with Vertex histogram as the base kernel
wl_kernel = WeisfeilerLehman(n_iter=5, normalize=True, base_graph_kernel=VertexHistogram)

In [ ]:
# compute Weisfeiler-Lehman kernel
K = wl_kernel.fit_transform(X_mutag)

#### Exercise

Compute the Kernel PCA embedding of MUTAG graph dataset, given by Weisfeiler-Lehman kernel.

In [ ]:
# apply Kernel PCA with Weisfeiler-Lehman kernel as precomputed kernel
kpca = #your code here
X_kpca_mutag = #your code here

In [ ]:
plt.figure(figsize=(6,6))

plt.title('Graph embedding w/ Kernel PCA')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid(linestyle="dotted")
plt.scatter(X_kpca_mutag[y_mutag==2, 0], X_kpca_mutag[y_mutag==2, 1], color='red', alpha=0.5)
plt.scatter(X_kpca_mutag[y_mutag==1, 0], X_kpca_mutag[y_mutag==1, 1], color='blue', alpha=0.5)

plt.show()

### 1.3. Swiss roll

In [ ]:
X, color = make_swiss_roll(n_samples=1000, random_state=123)

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=color, cmap=plt.cm.rainbow)
plt.title('Swiss Roll in 3D')
plt.show()

#### Exercise

Apply Kernel PCA with different kernels and kernel parameters to Swiss roll dataset. Compare results.

In [ ]:
# standartize the data
X_std = StandardScaler().fit_transform(X)

In [ ]:
kpca = KernelPCA(n_components=2, kernel="poly", degree=9)
X_kpca = kpca.fit_transform(X_std)

In [ ]:
plt.figure(figsize=(12,5.75))

plt.subplot(121)
plt.title('Original data')
plt.xlabel('X')
plt.ylabel('Z')
plt.grid(linestyle="dotted")
plt.scatter(X_std[:, 0], X_std[:, 2], c=color, cmap="Reds", alpha=0.25)

plt.subplot(122)
plt.title('Kernel PCA /w Gaussian kernel')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid(linestyle="dotted")
plt.scatter(X_kpca[:, 0], X_kpca[:, 1], c=color, cmap="Reds", alpha=0.25)

plt.show()

**NB:** Kernel PCA with standard kernels fails to unroll swiss roll dataset.

## 2. Manifold Learning Methods

### 2.1 PCA

In [ ]:
model = PCA(n_components=2)
X_pca = model.fit_transform(X)

In [ ]:
plt.figure(figsize=(6,6))
plt.grid(linestyle="dotted")
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=color, cmap=plt.cm.rainbow)

### 2.2 Kernel PCA

In [ ]:
model = KernelPCA(n_components=2, kernel="poly", degree=3)
X_kpca = model.fit_transform(X)

In [ ]:
plt.figure(figsize=(6,6))
plt.grid(linestyle="dotted")
plt.scatter(X_kpca[:, 0], X_kpca[:, 1], c=color, cmap=plt.cm.rainbow)

### 2.3 MDS

#### Intuition

Embedding should preserve distances between points.

$$J_{MDS}(z_1, \dots, z_n) = \sum_{ij}^n ((x_i, x_j) - (z_i, z_j))^2$$

#### Algorithm

Given a dataset $X = \{x_1, \dots, x_n \} \in \mathbb{R}^{D \times n}$,

1. Compute Gram matrix of $G \in \mathbb{R}^{n \times n}$, s.t. $g_{ij} = (x_i, x_j) = X^TX$
2. Find $d$ eigenvectors $\{ v_1, \dots, v_d \} \in \mathbb{R}^{n \times d}$ of $G$, corresponding to $d$ largest eigenvalues: $\Lambda = diag(\lambda_1, \dots, \lambda_d) \in \mathbb{R}^{d \times d}$.
3. Compute the embedding $\{z_1, \dots, z_n \} = \Lambda^{1/2} \{ v_1, \dots, v_d \}^T$

### 2.4 Isomap

#### Intuition

Manifold is globally isometric to a convex subset of Euclidean space.

#### Algorithm

Given a dataset $X = \{x_1, \dots, x_n \} \in \mathbb{R}^{D \times n}$,

1. Estimate the neighborhood $\mathcal{N}_i$ of each point, eigher within $\varepsilon$-ball or take $k$ nearest neighbors.
2. Build a graph on points, with adjancency matrix $A$, s.t. $a_{ij} = 1$ if the points are neighbors and $0$ otherwise.
3. Estimate pairwise geodesic distances between any two points, by shortest paths between nodes of the graph.
4. Perform MDS on pairwise distance matrix.

In [ ]:
model = Isomap(n_components=2)
X_isomap = model.fit_transform(X)

In [ ]:
plt.figure(figsize=(6,6))
plt.grid(linestyle="dotted")
plt.scatter(X_isomap[:, 0], X_isomap[:, 1], c=color, cmap=plt.cm.rainbow)

### 2.5 Locally Linear Embedding

#### Intuition

Manifold is locally linear. The same weights that reconstruct the datapoints in $D$ dimensions should reconstruct it in the manifold in $d$ dimensions.

#### Algorithm

Given a dataset $X = \{x_1, \dots, x_n \} \in \mathbb{R}^D$,

- Estimate the neighborhood $\mathcal{N}_i$ of each point, eigher within $\varepsilon$-ball or take $k$ nearest neighbors.
- 

$$W = \arg \min_W \sum_i^n \| x_i - \sum_j^n W_{ij} x_j \|^2, \\
s.t. \sum_j^n W_{ij} = 1, \forall i, \\
W_{ij} = 0, x_j \notin \mathcal{N}_i$$

- Given the weights W, find the embedded points:

$$\{z_1, \dots, z_n\} = \arg \min_{\{z_1, \dots, z_n\}} \sum_i^n \| z_i - \sum_i^n W_{ij} z_j \|^2 \\
s.t. \sum_i^n z_i = 0\\
cov(Z) = \mathbb{I}$$

The same weights that reconstruct the datapoints in $D$ dimensions should reconstruct it in the manifold in $d$ dimensions.

In [ ]:
model = LocallyLinearEmbedding(n_components=2, n_neighbors=15)
X_lle = model.fit_transform(X)

In [ ]:
plt.figure(figsize=(6,6))
plt.grid(linestyle="dotted")
plt.scatter(X_lle[:, 0], X_lle[:, 1], c=color, cmap=plt.cm.rainbow)

### 1.6 Laplacian Eigenmaps

#### Intuition

Manifold can be reconstructed with computing Laplacian of the graph, which is empirical version of Laplace-Beltrami operator of smooth manifold.

#### Algorithm

Given a dataset $X = \{x_1, \dots, x_n \} \in \mathbb{R}^{D \times n}$,

1. Estimate the neighborhood $\mathcal{N}_i$ of each point, eigher within $\varepsilon$-ball or take $k$ nearest neighbors.
2. Build adjancency graph, with adjacency matrix $A$ s.t.

$$a_{ij} = \exp(\lambda \| x_i - x_j \|^2), \textrm{if}~x_j \in \mathcal{N}_i \\
a_{ij} = 0,~\textrm{otherwise}$$

3. Compute graph Laplacian $L = D - A \in \mathbb{R}^{n \times n}$, where $D$ s.t. $d_{ii} = \sum_i^n A_{ij}$ and solve eigenvalue problem: 

$$Lv = \lambda Dv,\\
v \in \mathbb{R}^n$$

Given $d+1$ eigenvectors, corresponding to _smallest_ eigenvalues, compute the emdedding:

$$\{z_1, \dots, z_n \} = \{f_1(i), \dots, f_d(i)\}^T \in \mathbb{R}^d$$

In [ ]:
model = SpectralEmbedding(n_components=2, n_neighbors=25)
X_le = model.fit_transform(X)

In [ ]:
plt.figure(figsize=(6,6))
plt.grid(linestyle="dotted")
plt.scatter(X_le[:, 0], X_le[:, 1], c=color, cmap=plt.cm.rainbow)

### 2.7 Local Tangent Space Alignment

#### Intuition

Local linearity assumption. Manifold can be approximated using tangent spaces to each point.

#### Algorithm

Given a dataset $X = \{x_1, \dots, x_n \} \in \mathbb{R}^{D \times n}$,

1. Estimate the neighborhood $\mathcal{N}_i$ of each point, eigher within $\varepsilon$-ball or take $k$ nearest neighbors.
2. Build a graph on points, with adjancency matrix $A$, s.t. $a_{ij} = 1$ if the points are neighbors and $0$ otherwise.
3. Estimate tangent spaces to each point using Local PCA, align tangent spaces.
4. Compute the embedding.

In [ ]:
model = LocallyLinearEmbedding(n_components=2, n_neighbors=15, method="ltsa")
X_ltsa = model.fit_transform(X)

In [ ]:
plt.figure(figsize=(6,6))
plt.grid(linestyle="dotted")
plt.scatter(X_ltsa[:, 0], X_ltsa[:, 1], c=color, cmap=plt.cm.rainbow)

### 2.8 Evaluating the performance

To evaluate the performance of manifold learning algorithms implement neighborhood preserving ratio (NPR) metric, which is defined as follows:

$$NPR = \frac{1}{kn} \sum_{i=1}^n \left|\mathcal{N}_k(x_i) \bigcap \mathcal{N}_k(z_i) \right|,$$

where $n$ is the number of data points, $\mathcal{N}_k(x_i)$ is the set of $k$-nearest neighbors of data points $x_i$ of original dimension and $\mathcal{N}_k(z_i)$ is the set of $k$-nearest neighbors of $z_i$ of reduced dimension. $|\cdot|$ represents the number of intersection points.

**Exercise:** Evaluate manifold learning methods performance using NPR as a metric on swiss roll for 3 different values of $k$.

In [ ]:
def NPR(X, Z, k=21):
    
    # your code here
    
    return 

In [ ]:
NPR(X, X_kpca)

In [ ]:
data = [X_pca, X_kpca, X_isomap, X_lle, X_le, X_ltsa]

k = 21
n = X.shape[0]

for z in data:
    print(NPR(X, z))

### 2.9. Digits

In [ ]:
X, y = load_digits(return_X_y=True)
shape = (int(np.sqrt(X.shape[1])), int(np.sqrt(X.shape[1])))

In [ ]:
# plot numbers
_, ax = plt.subplots(10, 10, figsize=(6, 6),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw=dict(hspace=0.1, wspace=0.1))

for i, a in enumerate(ax.flat):
    a.imshow(X[i].reshape(shape), cmap=plt.cm.gray)

In [ ]:
# n components, k nearest neighbors
n, k = 2, 15

models = [
    ("PCA", PCA(n_components=n)),
    ("Isomap", Isomap(n_components=n, n_neighbors=k)),
    ("LLE", LocallyLinearEmbedding(n_components=n, n_neighbors=k)),
    ("Laplacian Eignenmaps", SpectralEmbedding(n_components=n, n_neighbors=k))
]

In [ ]:
plt.figure(figsize=(16,25))

for key, model in enumerate(models):
    X_transformed = MinMaxScaler().fit_transform(model[1].fit_transform(X))
    plt.subplot(320 + (key+1))
    plt.title(model[0])
    plt.grid(linestyle="dotted")
    plt.scatter(X_transformed[:, 0], X_transformed[:, 1], alpha=0.5, s=10, c=y, cmap=plt.cm.rainbow)
    
    # plot images
    ax = plt.gca()
    for i in range(0, X.shape[0], 10):
        box = offsetbox.AnnotationBbox(offsetbox.OffsetImage(X[i].reshape(shape), cmap="gray", zoom=1.5), X_transformed[i], frameon=False)
        ax.add_artist(box)
    
plt.show()

### 2.10 MNIST

In [ ]:
mnist = fetch_openml('mnist_784', version=1)
train_size = 60000

X_train, X_test, y_train, y_test = \
mnist.data[:train_size,:], mnist.data[train_size:,:], \
mnist.target[:train_size], mnist.target[train_size:]

print("Dataset summary:\nSamples: {}, features: {}, classes: {}"
      .format(X_train.shape[0] + X_test.shape[0], X_train.shape[1], np.unique(y_train).shape[0]))

**Exercise:** explore manifolds of different digits, try to interpret embedding coordinates.

In [ ]:
X = X_test[y_test.astype(int)==8]
shape = (int(np.sqrt(X.shape[1])), int(np.sqrt(X.shape[1])))

In [ ]:
# plot numbers
_, ax = plt.subplots(10, 10, figsize=(6, 6),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw=dict(hspace=0.1, wspace=0.1))

for i, a in enumerate(ax.flat):
    a.imshow(X[i].reshape(shape), cmap=plt.cm.gray)

In [ ]:
# n components, k nearest neighbors
n, k = 2, 15

models = [
    ("PCA", PCA(n_components=n)),
    ("Isomap", Isomap(n_components=n, n_neighbors=k)),
    ("LLE", LocallyLinearEmbedding(n_components=n, n_neighbors=k)),
    ("Laplacian Eignenmaps", SpectralEmbedding(n_components=n, n_neighbors=k))
]

In [ ]:
plt.figure(figsize=(16,25))

for key, model in enumerate(models):
    X_transformed = MinMaxScaler().fit_transform(model[1].fit_transform(X))
    plt.subplot(320 + (key+1))
    plt.title(model[0])
    plt.grid(linestyle="dotted")
    #plt.scatter(X_transformed[:, 0], X_transformed[:, 1], alpha=0.5, s=10)
    
    # plot images
    ax = plt.gca()
    for i in range(0, X.shape[0], 10):
        box = offsetbox.AnnotationBbox(offsetbox.OffsetImage(X[i].reshape(shape), cmap=plt.cm.get_cmap('gray'), zoom=0.75), X_transformed[i], frameon=False)
        ax.add_artist(box)
    
plt.show()

### 2.11 Airfoils

In [ ]:
ref_points = np.loadtxt('../seminar1/data/ref_points.csv', delimiter=',')
X = np.loadtxt('../seminar1/data/airfoils.csv', delimiter=',')
X_oos = np.loadtxt('../seminar1/data/test_afl.csv', delimiter=',').reshape(1, -1)
X.shape, X_oos.shape

In [ ]:
# n components, k nearest neighbors
n, k = 2, 15

models = [
    ("PCA", PCA(n_components=n)),
    ("Isomap", Isomap(n_components=n, n_neighbors=k)),
    ("LLE", LocallyLinearEmbedding(n_components=n, n_neighbors=k)),
    ("Laplacian Eignenmaps", SpectralEmbedding(n_components=n, n_neighbors=k))
]

In [ ]:
plt.figure(figsize=(16,25))

# color, proportional to height-to-width ratio
colors = MinMaxScaler().fit_transform((np.max(X[:, :], axis=1) - np.min(X[:, :], axis=1)).reshape(-1, 1)).ravel()

for key, value in enumerate(models):
    X_transformed = MinMaxScaler().fit_transform(value[1].fit_transform(X))
    plt.subplot(320 + (key+1))
    plt.title(value[0])
    plt.grid(linestyle="dotted")
    plt.scatter(X_transformed[:, 0], X_transformed[:, 1], c=colors, cmap="Reds")
    
    for i in range(0, 199, 10):
        plt.plot(X_transformed[i, 0] + ref_points / 4, X_transformed[i, 1] + X[i, :] / 2, '-', label = 'Airfoil #' + str(i), c='k', alpha=0.5)

plt.show()

## 3. Out-of-Sample extension

In [ ]:
model = LocallyLinearEmbedding(n_components=2, n_neighbors=21).fit(X)

X_transformed = model.transform(X)
X_transformed_oos = model.transform(X_oos)

In [ ]:
# scale data
scaler = MinMaxScaler().fit(X_transformed)
X_transformed = scaler.transform(X_transformed)
X_transformed_oos = scaler.transform(X_transformed_oos)

In [ ]:
plt.figure(figsize=(10,10))
plt.grid(linestyle="dotted")
plt.scatter(X_transformed[:, 0], X_transformed[:, 1], c=colors, cmap="Reds")
    
for i in range(0, 199, 10):
    plt.plot(X_transformed[i, 0] + ref_points / 4, X_transformed[i, 1] + X[i, :] / 2, '-', c='k', alpha=0.5)

plt.plot(X_transformed_oos[0, 0] + ref_points / 4, X_transformed_oos[0, 1] + X_oos[0, :] / 2, '-', c='b', alpha=0.5)

## 4. Non-Euclidean distances

In [ ]:
take_n = 500

X = np.load("./data/lm_3000.npy")
X = X[:take_n]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(16.75,3))
    
for i in range(5):
    ax[i].invert_yaxis()
    ax[i].set_title("Point cloud {}".format(i))
    ax[i].scatter(X[i,:,0], X[i,:,1], c="b", s=7)
        
plt.show()

### $p$-Wasserstein distance

Given 2 sets of vectors $\mathbf{a} \in \mathbb{R}^n$ and $\mathbf{b} \in \mathbb{R}^m$ (not nessesarily of same cardinality) the $p$-Wasserstein distance

\begin{equation}
    W_p^p := \min_{\mathbf{P} \in \mathbf{U}(\mathbf{a}, \mathbf{b})} \langle \mathbf{D}, \mathbf{P} \rangle_F,
\end{equation}

where $\mathbf{U}(\mathbf{a}, \mathbf{b}) := \{ P \in \mathbb{R}^{n \times m}_{\geq 0} \mid \mathbf{P} \mathbf{1}_m = \mathbf{a}, \mathbf{P}^T \mathbf{1}_n = \mathbf{b} \}$, and $\mathbf{D}$ is matrix of $p$-th powers of the _ground distance_ between all pairs of vectors of $\mathbf{a}$ and $\mathbf{b}$. For $\mathbb{R}^n$ it is the Euclidean distance.

In [ ]:
%%time
n, m, _ = X.shape
    
# distance matrix
D = np.zeros((n, n))

# uniform measures as points
a = np.ones(m) / m
b = np.ones(m) / m

for i in range(n):
    for j in range(i+1, n):

        # squared Euclidean distance as the ground metric
        M_ij = ot.dist(X[i], X[j], metric="sqeuclidean")

        # 2-Wasserstein distance, take square root
        D[i,j] = ot.emd2(a, b, M_ij) ** 0.5

D = D + D.T
D

In [ ]:
X_pc_mds = MDS(dissimilarity="precomputed", max_iter=1000, eps=1e-6, n_jobs=-1).fit_transform(D)
X_pc_le = Isomap(metric="precomputed", n_jobs=-1).fit_transform(D)

In [ ]:
# for visualization purpose only
a_mds = np.array([0.05, 0.05])
a_isomap = np.array([0.1, 0.05])
b_mds = np.array([-0.025, -0.027])
b_isomap = np.array([-0.05, -0.027])

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16.75,8))

ax[0].invert_yaxis()
ax[0].grid(linestyle="dotted")
ax[0].set_title("MDS w/ Wasserstein distance".format(i))
ax[0].scatter(X_pc_mds[:,0], X_pc_mds[:,1], c="grey", s=2)

for i in range(10):
    ax[0].scatter(X[i,:,0] * a_mds[0] + b_mds[0] + X_pc_mds[i,0], X[i,:,1] * a_mds[1] + b_mds[1] + X_pc_mds[i,1], s=7, c="b")
    ax[0].scatter(X_pc_mds[i,0], X_pc_mds[i,1], c="red", s=20)

ax[1].invert_yaxis()
ax[1].grid(linestyle="dotted")
ax[1].set_title("Isomap w/ Wasserstein distance".format(i))
ax[1].scatter(X_pc_le[:,0], X_pc_le[:,1], c="grey", s=2)

for i in range(10):
    ax[1].scatter(X[i,:,0] * a_isomap[0] + b_isomap[0] + X_pc_le[i,0], X[i,:,1] * a_isomap[1] + b_isomap[1] + X_pc_le[i,1], s=7, c="b")
    ax[1].scatter(X_pc_le[i,0], X_pc_le[i,1], c="red", s=20)
        
plt.show()